In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicQgsTensor, SymbolicQgsTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _shift_dict_keys, _add_to_dict

In [ ]:
model_parameters = QgParams({'n': 1.5}, dynamic_T=True)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
# Setting MAOOAM parameters according to the publication linked above
model_parameters.set_params({'kd': 0.0290, 'kdp': 0.0290, 'r': 1.e-7,
                             'h': 136.5, 'd': 1.1e-7})
model_parameters.atemperature_params.set_params({'eps': 0.7, 'hlambda': 15.06})
model_parameters.gotemperature_params.set_params({'gamma': 5.6e8})

In [ ]:
model_parameters.atemperature_params.set_insolation(103., 0)
model_parameters.atemperature_params.set_insolation(103., 1)
model_parameters.gotemperature_params.set_insolation(310., 0)
model_parameters.gotemperature_params.set_insolation(310., 1)

In [ ]:
model_parameters.print_params()

## Testing the symbolic_qgtensor

In [ ]:
from qgs.functions.symbolic_tendencies import create_symbolic_equations, translate_equations, equation_as_function

In [ ]:
f_num, Df, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
%%time
funcs, ten = create_symbolic_equations(model_parameters, continuation_variables=[], return_symbolic_qgtensor=True, language='python')

In [ ]:
print(funcs)

In [ ]:
ten.print_tensor(ten_num.tensor.todense())

In [ ]:
from qgs.functions.symbolic_mul import symbolic_sparse_mult5
from qgs.functions.symbolic_tendencies import equation_as_function

In [ ]:
xx = list()
xx.append(1)
for i in range(1, 39):
    xx.append(sy.Symbol('U[' + str(i-1) + ']'))

In [ ]:
fun_out = symbolic_sparse_mult5(ten.tensor_dic, xx, xx, xx, xx)

In [ ]:
test_fun = '\n'.join(equation_as_function(fun_out, model_parameters))

In [ ]:
print(test_fun)

In [ ]:
def arr_to_dict(arr):
    x, y, z, w, s = arr.shape
    output = dict()
    for i in range(x):
        for j in range(y):
            for k in range(z):
                for l in range(w):
                    for m in range(s):
                        if arr[i, j, k, l, m] != 0:
                            output[(i, j, k, l, m)] = arr[i, j, k, l, m]
    return output

In [ ]:
ten_num_dic = arr_to_dict(ten_num.tensor.todense())

In [ ]:
ten_dic = ten.sub_tensor()

In [ ]:
ten_dic

In [ ]:
ten_num_dic

In [ ]:
for k in ten_num_dic.keys():
    if ten_dic[k] - ten_num_dic[k] != 0:
        print(k, (ten_dic[k] - ten_num_dic[k]) / ten_num_dic[k])

In [ ]:
test_eq = symbolic_sparse_mult5(ten_num_dic, xx, xx, xx, xx)

In [ ]:
import sparse as sp

In [ ]:
subbed_tensor = ten.sub_tensor()
dims = (model_parameters.ndim + 1, model_parameters.ndim + 1, model_parameters.ndim + 1, model_parameters.ndim + 1, model_parameters.ndim + 1)
coords = np.array([list(k) for k in subbed_tensor.keys()]).T
data = np.array(list(subbed_tensor.values()), dtype=float)
subbed_tensor_sp = sp.COO(coords, data, shape=dims)

In [ ]:
num_t = ten_num.tensor.todense()

In [ ]:
err = subbed_tensor_sp.todense() - num_t

In [ ]:
for i in range(model_parameters.ndim + 1):
    for j in range(model_parameters.ndim + 1):
        for k in range(model_parameters.ndim + 1):
            for ell in range(model_parameters.ndim + 1):
                for m in range(model_parameters.ndim + 1):
                    if abs(err[i, j, k, ell, m]) > 1e-15:
                        if num_t[i, j, k, ell, m] != 0:
                            print(i, j, k, ell, m, err[i, j, k, ell, m])


In [ ]:
ten.print_tensor(num_t)

In [ ]:
from numba import njit

In [ ]:
@njit
def f(t, U):
	#Tendency function of the qgs model
	F = np.empty_like(U)
	F[0] = -0.0145*U[0] + 0.0145*U[11] + 0.0505574149661188*U[22] + 0.0734121368001177*U[24]
	F[1] = -1.24659182237138*U[0]*U[2] - 1.24659182237138*U[11]*U[13] + 0.0145*U[12] - 1.9945469157942*U[14]*U[16] - 2.59615384615385*U[15]*U[18] + 2.59615384615385*U[16]*U[17] - 0.0145*U[1] - 0.00295864958311857*U[21] + 0.115315741885204*U[2] - 1.9945469157942*U[3]*U[5] - 2.59615384615385*U[4]*U[7] + 2.59615384615385*U[5]*U[6]
	F[2] = 1.24659182237138*U[0]*U[1] + 1.24659182237138*U[11]*U[12] + 0.0145*U[13] + 1.9945469157942*U[14]*U[15] + 2.59615384615385*U[15]*U[17] + 2.59615384615385*U[16]*U[18] - 0.115315741885204*U[1] + 0.0145*U[25] - 0.0145*U[2] + 1.9945469157942*U[3]*U[4] + 2.59615384615385*U[4]*U[6] + 2.59615384615385*U[5]*U[7]
	F[3] = 2.16075915877706*U[12]*U[16] - 2.16075915877706*U[13]*U[15] + 0.0145*U[14] + 4.32151831755411*U[17]*U[20] - 4.32151831755411*U[18]*U[19] + 2.16075915877706*U[1]*U[5] - 0.00216427290094687*U[21] + 0.0238416302768307*U[23] - 2.16075915877706*U[2]*U[4] - 0.0145*U[3] + 4.32151831755411*U[6]*U[9] - 4.32151831755411*U[7]*U[8]
	F[4] = -1.21002512891515*U[0]*U[5] - 1.21002512891515*U[11]*U[16] + 2.43*U[12]*U[18] + 0.345721465404329*U[13]*U[14] - 2.43*U[13]*U[17] + 0.0145*U[15] + 2.43*U[1]*U[7] - 0.00449241352700723*U[22] + 0.345721465404329*U[2]*U[3] - 2.43*U[2]*U[6] - 0.0145*U[4] + 0.059964185780306*U[5]
	F[5] = 1.21002512891515*U[0]*U[4] + 1.21002512891515*U[11]*U[15] - 0.345721465404329*U[12]*U[14] - 2.43*U[12]*U[17] - 2.43*U[13]*U[18] + 0.0145*U[16] - 0.345721465404329*U[1]*U[3] - 2.43*U[1]*U[6] + 0.0145*U[26] - 2.43*U[2]*U[7] - 0.059964185780306*U[4] - 0.0145*U[5]
	F[6] = -3.24113873816558*U[0]*U[7] - 3.24113873816558*U[11]*U[18] + 0.675*U[12]*U[16] + 0.675*U[13]*U[15] - 5.18582198106493*U[14]*U[20] + 0.0145*U[17] + 0.675*U[1]*U[5] - 0.000192312222902707*U[21] + 0.675*U[2]*U[4] - 5.18582198106493*U[3]*U[9] - 0.0145*U[6] + 0.0749552322253824*U[7]
	F[7] = 3.24113873816558*U[0]*U[6] + 3.24113873816558*U[11]*U[17] - 0.675*U[12]*U[15] + 0.675*U[13]*U[16] + 5.18582198106493*U[14]*U[19] + 0.0145*U[18] - 0.675*U[1]*U[4] + 0.675*U[2]*U[5] + 5.18582198106493*U[3]*U[8] - 0.0749552322253824*U[6] - 0.0145*U[7]
	F[8] = -2.65939588772561*U[0]*U[9] - 2.65939588772561*U[11]*U[20] - 2.65939588772561*U[14]*U[18] + 0.0145*U[19] - 0.000431962839135312*U[22] - 2.65939588772561*U[3]*U[7] - 0.0145*U[8] + 0.0576578709426019*U[9]
	F[9] = 2.6593958877256*U[0]*U[8] + 2.6593958877256*U[11]*U[19] + 2.6593958877256*U[14]*U[17] + 0.0145*U[20] + 2.6593958877256*U[3]*U[6] - 0.0576578709426019*U[8] - 0.0145*U[9]
	F[10] = -0.000510753549879668*U[10]**4 - 0.014593023255814*U[10] - 0.00183644645351737*U[21] - 0.00374635076517544*U[23] + 1.59610484337396e-5*U[29]**4 + 5.17501542233338e-5*U[29]**3*U[30] + 1.72500514077779e-5*U[29]**3*U[32] + 0.00729651162790698*U[29] + 0.00591432955679693*U[30] + 0.00197144318559898*U[32] + 0.000531003677198668
	F[11] = 0.00131818181818182*U[0] - 0.00185728563592607*U[10]**3*U[11] - 0.0198572938689218*U[11] - 1.63693875664928*U[12]*U[2] + 1.63693875664928*U[13]*U[1] - 1.30955100531943*U[15]*U[5] + 1.30955100531943*U[16]*U[4] - 3.27387751329857*U[17]*U[7] + 3.27387751329857*U[18]*U[6] - 2.61910201063885*U[19]*U[9] + 2.61910201063885*U[20]*U[8] - 0.00459612863328353*U[22] - 0.00667383061819252*U[24] + 4.43550120954776e-5*U[29]**3*U[31] + 1.7742004838191e-5*U[29]**3*U[33] + 0.00506916671004016*U[31] + 0.00202766668401606*U[33] + 0.000482730615635153
	F[12] = -1.6647358298754*U[0]*U[13] - 0.0015418975090707*U[10]**3*U[12] + 1.05320021890077*U[11]*U[2] - 0.0287966213251426*U[12] + 0.0282849932925971*U[13] + 1.68512035024122*U[14]*U[5] + 1.06132075471698*U[15]*U[7] - 2.66357732780065*U[16]*U[3] - 1.06132075471698*U[16]*U[6] + 2.33490566037736*U[17]*U[5] - 2.33490566037736*U[18]*U[4] + 0.0035566037735849*U[1] + 0.000725706501519649*U[21] - 2.04500508591402e-5*U[29]**3*U[30] - 0.00233715902975374*U[30]
	F[13] = 1.6647358298754*U[0]*U[12] - 0.0015418975090707*U[10]**3*U[13] - 1.05320021890077*U[11]*U[1] - 0.0282849932925971*U[12] - 0.0287966213251426*U[13] - 1.68512035024122*U[14]*U[4] + 2.66357732780065*U[15]*U[3] - 1.06132075471698*U[15]*U[6] - 1.06132075471698*U[16]*U[7] + 2.33490566037736*U[17]*U[4] + 2.33490566037736*U[18]*U[5] - 0.0035566037735849*U[25] + 4.81842971584593e-5*U[29]**3*U[34] + 0.0035566037735849*U[2] + 0.00550680122860904*U[34]
	F[14] = -0.00145929585679905*U[10]**3*U[14] - 1.44050610585137*U[12]*U[5] + 1.44050610585137*U[13]*U[4] - 0.0311378737541528*U[14] - 2.67522562515254*U[15]*U[2] + 2.67522562515254*U[16]*U[1] - 2.88101221170274*U[17]*U[9] + 2.88101221170274*U[18]*U[8] - 5.35045125030509*U[19]*U[7] + 5.35045125030509*U[20]*U[6] + 0.000618363685984819*U[21] - 0.00681189436480877*U[23] - 1.74251833232233e-5*U[29]**3*U[30] + 3.1365329981802e-5*U[29]**3*U[32] - 0.00199145835037292*U[30] + 0.00358462503067126*U[32] + 0.00414285714285714*U[3]
	F[15] = -1.35185957626052*U[0]*U[16] - 0.00125723950739611*U[10]**3*U[15] + 0.421071015556554*U[11]*U[5] - 0.449999999999999*U[12]*U[7] - 1.63996079743079*U[13]*U[3] + 0.45*U[13]*U[6] + 1.90590038620335*U[14]*U[2] - 0.0368649373881932*U[15] + 0.0230631483770408*U[16] - 2.31923076923077*U[17]*U[2] + 2.31923076923077*U[18]*U[1] + 0.00172785135654124*U[22] - 1.66746568543758e-5*U[29]**3*U[31] - 0.00190568351656843*U[31] + 0.00557692307692308*U[4]
	F[16] = 1.35185957626052*U[0]*U[15] - 0.00125723950739611*U[10]**3*U[16] - 0.421071015556554*U[11]*U[4] + 1.63996079743079*U[12]*U[3] + 0.45*U[12]*U[6] + 0.45*U[13]*U[7] - 1.90590038620335*U[14]*U[1] - 0.0230631483770408*U[15] - 0.0368649373881932*U[16] - 2.31923076923077*U[17]*U[1] - 2.31923076923077*U[18]*U[2] - 0.00557692307692308*U[26] + 3.92887346061283e-5*U[29]**3*U[35] + 0.00449016100178891*U[35] + 0.00557692307692308*U[5]
	F[17] = -3.421202001397*U[0]*U[18] - 0.00102150709975934*U[10]**3*U[17] + 0.180063263231421*U[11]*U[7] - 0.7875*U[12]*U[5] - 0.7875*U[13]*U[4] + 0.288101221170274*U[14]*U[9] + 1.4625*U[15]*U[2] + 1.4625*U[16]*U[1] - 0.043546511627907*U[17] + 0.0374776161126912*U[18] - 5.47392320223521*U[20]*U[3] + 9.61561114513536e-5*U[21] - 2.70963173883608e-6*U[29]**3*U[30] - 0.00030967357144237*U[30] + 0.00725*U[6]
	F[18] = 3.421202001397*U[0]*U[17] - 0.00102150709975934*U[10]**3*U[18] - 0.180063263231421*U[11]*U[6] + 0.7875*U[12]*U[4] - 0.7875*U[13]*U[5] - 0.288101221170275*U[14]*U[8] - 1.4625*U[15]*U[1] + 1.4625*U[16]*U[2] - 0.0374776161126912*U[17] - 0.043546511627907*U[18] + 5.47392320223521*U[19]*U[3] + 0.00725*U[7]
	F[19] = -2.75575081119392*U[0]*U[20] - 0.000888267043268989*U[10]**3*U[19] - 0.25052280101763*U[11]*U[9] + 1.00209120407052*U[14]*U[7] - 4.00836481628207*U[18]*U[3] - 0.0473230535894843*U[19] + 0.0325892314023402*U[20] + 0.00024415290907648*U[22] - 2.35620151203137e-6*U[29]**3*U[31] - 0.000269281366471626*U[31] + 0.00819565217391304*U[8]
	F[20] = 2.75575081119392*U[0]*U[19] - 0.000888267043268989*U[10]**3*U[20] + 0.250522801017629*U[11]*U[8] - 1.00209120407052*U[14]*U[6] + 4.00836481628207*U[17]*U[3] - 0.0325892314023402*U[19] - 0.0473230535894843*U[20] + 0.00819565217391304*U[9]
	F[21] = 2.30554431681946e-7*U[12] + 2.5547312812995e-7*U[14] + 1.41879650265813e-7*U[17] - 2.30554431681946e-7*U[1] - 4.98594961275655e-7*U[21] - 0.000231547500016414*U[22]*U[25] - 0.000393263214313593*U[22]*U[27] - 0.000779175714340951*U[23]*U[26] - 0.00102174928578672*U[23]*U[28] - 0.00156202678582502*U[24]*U[27] - 2.49430228227487e-5*U[25] - 2.5547312812995e-7*U[3] - 1.41879650265813e-7*U[6]
	F[22] = 1.27676499203106e-7*U[0] - 1.27676499203106e-7*U[11] + 4.43165421807242e-7*U[15] + 1.84356815471813e-7*U[19] - 0.000297563940911795*U[21]*U[25] + 0.00056941247952257*U[21]*U[27] - 1.45521268835226e-6*U[22] - 0.00185518259457354*U[23]*U[25] - 0.00363689261114416*U[24]*U[26] - 2.49312940023328e-5*U[26] - 4.43165421807242e-7*U[4] - 1.84356815471813e-7*U[8]
	F[23] = -4.59275459509596e-7*U[14] - 0.00110122563125871*U[21]*U[26] + 0.0019601816236405*U[21]*U[28] + 0.000385428970940549*U[22]*U[25] - 3.04757790976247e-6*U[23] - 0.00547309138735578*U[24]*U[25] - 2.49117704610167e-5*U[27] + 4.59275459509596e-7*U[3]
	F[24] = 5.09747214298138e-8*U[0] - 5.09747214298138e-8*U[11] - 0.0028417174170155*U[21]*U[27] - 0.000594010608460015*U[22]*U[26] + 0.00259237963074834*U[23]*U[25] - 5.2727039845599e-6*U[24] - 2.48844888171634e-5*U[28]
	F[25] = -5.43087366736076e-7*U[13] + 0.000529111412123567*U[21]*U[22] + 2.49364240037345e-5*U[21] + 0.00146975392256546*U[22]*U[23] + 0.00288071768822831*U[23]*U[24] - 1.03680315467796e-6*U[25] + 5.43087366736076e-7*U[2]
	F[26] = -1.04390781015316e-6*U[16] + 0.00188040062849988*U[21]*U[23] + 0.00423090141412473*U[22]*U[24] + 2.49247013868922e-5*U[22] - 1.99291491029239e-6*U[26] + 1.04390781015316e-6*U[5]
	F[27] = -0.000176149545706673*U[21]*U[22] + 0.00440373864266684*U[21]*U[24] + 2.49051881654425e-5*U[23] - 3.58443843007766e-6*U[27]
	F[28] = -0.000938435679208949*U[21]*U[23] + 2.4877920928742e-5*U[24] - 5.80838943890229e-6*U[28]
	F[29] = 9.12059910499408e-6*U[10]**4 + 4.64878611115251e-5*U[10]**3*U[12] + 1.67415119447806e-5*U[10]**3*U[14] + 9.29757222230503e-6*U[10]**3*U[17] + 0.000521179401993355*U[10] + 0.000664115245477357*U[12] + 0.000239165516524778*U[14] + 0.000132823049095472*U[17] + 1.50119582264922*U[21]*U[35] - 0.750597911324608*U[21]*U[37] - 1.50119582264922*U[22]*U[34] + 1.12589686698691*U[22]*U[36] - 4.50358746794765*U[23]*U[35] + 2.25179373397383*U[23]*U[37] - 2.62709268963613*U[24]*U[34] - 5.62948433493457*U[24]*U[36] + 1.50119582264922*U[25]*U[31] + 2.62709268963613*U[25]*U[33] - 1.50119582264922*U[26]*U[30] + 4.50358746794765*U[26]*U[32] - 1.12589686698691*U[27]*U[31] + 5.62948433493457*U[27]*U[33] + 0.750597911324608*U[28]*U[30] - 2.25179373397383*U[28]*U[32] - 8.14339205803043e-7*U[29]**4 - 1.44655701782e-21*U[29]**3*U[30] - 0.000260589700996678*U[29] + 5.70773716822424e-5
	F[30] = -5.06294956237001e-22*U[10]**4 - 5.31652508387687e-5*U[10]**3*U[12] - 2.75103051082387e-5*U[10]**3*U[14] - 1.06330501677538e-5*U[10]**3*U[17] - 2.89312686026697e-20*U[10] - 0.000759506950146621*U[12] - 0.000393006100803065*U[14] - 0.000151901390029324*U[17] - 1.21682350103801*U[21]*U[35] + 0.608411750519007*U[21]*U[37] + 2.34182350103801*U[22]*U[34] - 1.28761762577851*U[22]*U[36] + 5.15047050311404*U[23]*U[35] - 2.57523525155702*U[23]*U[37] + 2.12944112681652*U[24]*U[34] + 6.43808812889255*U[24]*U[36] - 2.34182350103801*U[25]*U[31] - 2.12944112681652*U[25]*U[33] + 1.21682350103801*U[26]*U[30] - 5.15047050311404*U[26]*U[32] + 1.28761762577851*U[27]*U[31] - 6.43808812889255*U[27]*U[33] - 0.608411750519007*U[28]*U[30] + 2.57523525155702*U[28]*U[32] + 4.52049068068751e-23*U[29]**4 - 3.25735682321217e-6*U[29]**3*U[30] - 3.61639254455001e-22*U[29]**3*U[32] - 0.000260589700996678*U[30] - 3.16843061133567e-21
	F[31] = 2.78802933171573e-5*U[10]**3*U[11] - 1.54836099362061e-5*U[10]**3*U[15] - 3.09672198724123e-6*U[10]**3*U[19] + 0.000398291670074584*U[11] - 0.000221195408173122*U[15] - 4.42390816346244e-5*U[19] - 1.125*U[21]*U[34] + 0.375*U[21]*U[36] + 1.875*U[23]*U[34] + 2.25*U[24]*U[35] + 1.125*U[25]*U[30] - 1.875*U[25]*U[32] - 2.25*U[26]*U[33] - 0.375*U[27]*U[30] - 3.25735682321217e-6*U[29]**3*U[31] - 0.000260589700996678*U[31] + 4.36192251724445e-5
	F[32] = -5.06294956237001e-22*U[10]**4 - 1.25605469675209e-5*U[10]**3*U[12] + 2.05688778355549e-5*U[10]**3*U[14] - 2.51210939350418e-6*U[10]**3*U[17] - 2.89312686026697e-20*U[10] - 0.000179437180657833*U[12] + 0.000293842414478535*U[14] - 3.58874361315667e-5*U[17] - 1.90560783367934*U[21]*U[35] + 0.952803916839669*U[21]*U[37] - 1.46939216632066*U[22]*U[34] - 0.304205875259504*U[22]*U[36] + 1.21682350103801*U[23]*U[35] - 0.608411750519007*U[23]*U[37] + 3.33481370893884*U[24]*U[34] + 1.52102937629752*U[24]*U[36] + 1.46939216632066*U[25]*U[31] - 3.33481370893884*U[25]*U[33] + 1.90560783367934*U[26]*U[30] - 1.21682350103801*U[26]*U[32] + 0.304205875259504*U[27]*U[31] - 1.52102937629752*U[27]*U[33] - 0.952803916839669*U[28]*U[30] + 0.608411750519007*U[28]*U[32] + 4.52049068068751e-23*U[29]**4 + 3.61639254455001e-22*U[29]**3*U[30] - 3.25735682321217e-6*U[29]**3*U[32] - 0.000260589700996678*U[32] - 3.16843061133567e-21
	F[33] = 1.11521173268629e-5*U[10]**3*U[11] + 0.000159316668029834*U[11] - 1.875*U[21]*U[36] - 2.25*U[22]*U[35] - 2.625*U[23]*U[34] + 2.625*U[25]*U[32] + 2.25*U[26]*U[31] + 1.875*U[27]*U[30] - 3.25735682321217e-6*U[29]**3*U[33] - 0.000260589700996678*U[33] + 1.74476900689778e-5
	F[34] = 3.64823964199763e-5*U[10]**3*U[13] + 0.000521179401993355*U[13] + 1.125*U[21]*U[31] - 1.125*U[22]*U[30] + 1.875*U[22]*U[32] - 1.875*U[23]*U[31] + 2.625*U[23]*U[33] - 2.625*U[24]*U[32] - 3.25735682321217e-6*U[29]**3*U[34] - 0.000260589700996678*U[34]
	F[35] = 3.64823964199763e-5*U[10]**3*U[16] + 0.000521179401993355*U[16] + 1.5*U[21]*U[32] + 2.25*U[22]*U[33] - 1.5*U[23]*U[30] - 2.25*U[24]*U[31] - 3.25735682321217e-6*U[29]**3*U[35] - 0.000260589700996678*U[35]
	F[36] = -0.375*U[21]*U[31] + 1.875*U[21]*U[33] + 0.375*U[22]*U[30] - 1.875*U[24]*U[30] - 3.25735682321217e-6*U[29]**3*U[36] - 0.000260589700996678*U[36]
	F[37] = -0.75*U[21]*U[32] + 0.75*U[23]*U[30] - 3.25735682321217e-6*U[29]**3*U[37] - 0.000260589700996678*U[37]
	return F

In [ ]:
@njit
def f_test(t, U):
	#Tendency function of the qgs model
	F = np.empty_like(U)
	F[0] = -0.0145*U[0] + 0.0145*U[11] + 0.0505574149661188*U[22] + 0.0734121368001177*U[24]
	F[1] = -1.24659182237138*U[0]*U[2] - 1.24659182237138*U[11]*U[13] + 0.0145*U[12] - 1.9945469157942*U[14]*U[16] - 2.59615384615385*U[15]*U[18] + 2.59615384615385*U[16]*U[17] - 0.0145*U[1] - 0.00295864958311857*U[21] + 0.115315741885204*U[2] - 1.9945469157942*U[3]*U[5] - 2.59615384615385*U[4]*U[7] + 2.59615384615385*U[5]*U[6]
	F[2] = 1.24659182237138*U[0]*U[1] + 1.24659182237138*U[11]*U[12] + 0.0145*U[13] + 1.9945469157942*U[14]*U[15] + 2.59615384615385*U[15]*U[17] + 2.59615384615385*U[16]*U[18] - 0.115315741885204*U[1] + 0.0145*U[25] - 0.0145*U[2] + 1.9945469157942*U[3]*U[4] + 2.59615384615385*U[4]*U[6] + 2.59615384615385*U[5]*U[7]
	F[3] = 2.16075915877706*U[12]*U[16] - 2.16075915877706*U[13]*U[15] + 0.0145*U[14] + 4.32151831755411*U[17]*U[20] - 4.32151831755411*U[18]*U[19] + 2.16075915877706*U[1]*U[5] - 0.00216427290094687*U[21] + 0.0238416302768307*U[23] - 2.16075915877706*U[2]*U[4] - 0.0145*U[3] + 4.32151831755411*U[6]*U[9] - 4.32151831755411*U[7]*U[8]
	F[4] = -1.21002512891515*U[0]*U[5] - 1.21002512891515*U[11]*U[16] + 2.43*U[12]*U[18] + 0.345721465404329*U[13]*U[14] - 2.43*U[13]*U[17] + 0.0145*U[15] + 2.43*U[1]*U[7] - 0.00449241352700723*U[22] + 0.345721465404329*U[2]*U[3] - 2.43*U[2]*U[6] - 0.0145*U[4] + 0.059964185780306*U[5]
	F[5] = 1.21002512891515*U[0]*U[4] + 1.21002512891515*U[11]*U[15] - 0.345721465404329*U[12]*U[14] - 2.43*U[12]*U[17] - 2.43*U[13]*U[18] + 0.0145*U[16] - 0.345721465404329*U[1]*U[3] - 2.43*U[1]*U[6] + 0.0145*U[26] - 2.43*U[2]*U[7] - 0.059964185780306*U[4] - 0.0145*U[5]
	F[6] = -3.24113873816558*U[0]*U[7] - 3.24113873816558*U[11]*U[18] + 0.675*U[12]*U[16] + 0.675*U[13]*U[15] - 5.18582198106493*U[14]*U[20] + 0.0145*U[17] + 0.675*U[1]*U[5] - 0.000192312222902707*U[21] + 0.675*U[2]*U[4] - 5.18582198106493*U[3]*U[9] - 0.0145*U[6] + 0.0749552322253824*U[7]
	F[7] = 3.24113873816558*U[0]*U[6] + 3.24113873816558*U[11]*U[17] - 0.675*U[12]*U[15] + 0.675*U[13]*U[16] + 5.18582198106493*U[14]*U[19] + 0.0145*U[18] - 0.675*U[1]*U[4] + 0.675*U[2]*U[5] + 5.18582198106493*U[3]*U[8] - 0.0749552322253824*U[6] - 0.0145*U[7]
	F[8] = -2.65939588772561*U[0]*U[9] - 2.65939588772561*U[11]*U[20] - 2.6593958877256*U[14]*U[18] + 0.0145*U[19] - 0.000431962839135312*U[22] - 2.6593958877256*U[3]*U[7] - 0.0145*U[8] + 0.0576578709426019*U[9]
	F[9] = 2.6593958877256*U[0]*U[8] + 2.6593958877256*U[11]*U[19] + 2.6593958877256*U[14]*U[17] + 0.0145*U[20] + 2.6593958877256*U[3]*U[6] - 0.0576578709426019*U[8] - 0.0145*U[9]
	F[10] = -0.000510753549879668*U[10]**4 - 0.014593023255814*U[10] - 0.00183644645351737*U[21] - 0.00374635076517544*U[23] + 1.59610484337396e-5*U[29]**4 + 5.17501542233338e-5*U[29]**3*U[30] + 1.72500514077779e-5*U[29]**3*U[32] + 0.00729651162790698*U[29] + 0.00591432955679693*U[30] + 0.00197144318559898*U[32] + 0.000531003677198668
	F[11] = 0.00131818181818182*U[0] - 0.00185728563592607*U[10]**3*U[11] - 0.0198572938689218*U[11] - 1.63693875664928*U[12]*U[2] + 1.63693875664928*U[13]*U[1] - 1.30955100531943*U[15]*U[5] + 1.30955100531943*U[16]*U[4] - 3.27387751329857*U[17]*U[7] + 3.27387751329857*U[18]*U[6] - 2.61910201063885*U[19]*U[9] + 2.61910201063885*U[20]*U[8] - 0.00459612863328353*U[22] - 0.00667383061819252*U[24] + 4.43550120954775e-5*U[29]**3*U[31] + 1.7742004838191e-5*U[29]**3*U[33] + 0.00506916671004016*U[31] + 0.00202766668401607*U[33] + 0.000482730615635153
	F[12] = -1.6647358298754*U[0]*U[13] - 0.0015418975090707*U[10]**3*U[12] + 1.05320021890077*U[11]*U[2] - 0.0287966213251426*U[12] + 0.0282849932925971*U[13] + 1.68512035024122*U[14]*U[5] + 1.06132075471698*U[15]*U[7] - 2.66357732780065*U[16]*U[3] - 1.06132075471698*U[16]*U[6] + 2.33490566037736*U[17]*U[5] - 2.33490566037736*U[18]*U[4] + 0.00355660377358491*U[1] + 0.000725706501519649*U[21] - 2.04500508591402e-5*U[29]**3*U[30] - 0.00233715902975374*U[30]
	F[13] = 1.6647358298754*U[0]*U[12] - 0.0015418975090707*U[10]**3*U[13] - 1.05320021890077*U[11]*U[1] - 0.0282849932925971*U[12] - 0.0287966213251426*U[13] - 1.68512035024122*U[14]*U[4] + 2.66357732780065*U[15]*U[3] - 1.06132075471698*U[15]*U[6] - 1.06132075471698*U[16]*U[7] + 2.33490566037736*U[17]*U[4] + 2.33490566037736*U[18]*U[5] - 0.00355660377358491*U[25] + 4.81842971584593e-5*U[29]**3*U[34] + 0.00355660377358491*U[2] + 0.00550680122860904*U[34]
	F[14] = -0.00145929585679905*U[10]**3*U[14] - 1.44050610585137*U[12]*U[5] + 1.44050610585137*U[13]*U[4] - 0.0311378737541528*U[14] - 2.67522562515255*U[15]*U[2] + 2.67522562515255*U[16]*U[1] - 2.88101221170274*U[17]*U[9] + 2.88101221170274*U[18]*U[8] - 5.35045125030509*U[19]*U[7] + 5.35045125030509*U[20]*U[6] + 0.000618363685984819*U[21] - 0.00681189436480877*U[23] - 1.74251833232233e-5*U[29]**3*U[30] + 3.1365329981802e-5*U[29]**3*U[32] - 0.00199145835037292*U[30] + 0.00358462503067126*U[32] + 0.00414285714285714*U[3]
	F[15] = -1.35185957626052*U[0]*U[16] - 0.00125723950739611*U[10]**3*U[15] + 0.421071015556554*U[11]*U[5] - 0.45*U[12]*U[7] - 1.63996079743079*U[13]*U[3] + 0.45*U[13]*U[6] + 1.90590038620335*U[14]*U[2] - 0.0368649373881932*U[15] + 0.0230631483770408*U[16] - 2.31923076923077*U[17]*U[2] + 2.31923076923077*U[18]*U[1] + 0.00172785135654124*U[22] - 1.66746568543758e-5*U[29]**3*U[31] - 0.00190568351656843*U[31] + 0.00557692307692308*U[4]
	F[16] = 1.35185957626052*U[0]*U[15] - 0.00125723950739611*U[10]**3*U[16] - 0.421071015556554*U[11]*U[4] + 1.63996079743079*U[12]*U[3] + 0.45*U[12]*U[6] + 0.45*U[13]*U[7] - 1.90590038620335*U[14]*U[1] - 0.0230631483770408*U[15] - 0.0368649373881932*U[16] - 2.31923076923077*U[17]*U[1] - 2.31923076923077*U[18]*U[2] - 0.00557692307692308*U[26] + 3.92887346061283e-5*U[29]**3*U[35] + 0.00449016100178891*U[35] + 0.00557692307692308*U[5]
	F[17] = -3.421202001397*U[0]*U[18] - 0.00102150709975934*U[10]**3*U[17] + 0.180063263231421*U[11]*U[7] - 0.7875*U[12]*U[5] - 0.7875*U[13]*U[4] + 0.288101221170274*U[14]*U[9] + 1.4625*U[15]*U[2] + 1.4625*U[16]*U[1] - 0.043546511627907*U[17] + 0.0374776161126912*U[18] - 5.47392320223521*U[20]*U[3] + 9.61561114513536e-5*U[21] - 2.70963173883608e-6*U[29]**3*U[30] - 0.000309673571442371*U[30] + 0.00725*U[6]
	F[18] = 3.421202001397*U[0]*U[17] - 0.00102150709975934*U[10]**3*U[18] - 0.180063263231421*U[11]*U[6] + 0.7875*U[12]*U[4] - 0.7875*U[13]*U[5] - 0.288101221170275*U[14]*U[8] - 1.4625*U[15]*U[1] + 1.4625*U[16]*U[2] - 0.0374776161126912*U[17] - 0.043546511627907*U[18] + 5.4739232022352*U[19]*U[3] + 0.00725*U[7]
	F[19] = -2.75575081119392*U[0]*U[20] - 0.000888267043268988*U[10]**3*U[19] - 0.250522801017629*U[11]*U[9] + 1.00209120407052*U[14]*U[7] - 4.00836481628207*U[18]*U[3] - 0.0473230535894843*U[19] + 0.0325892314023402*U[20] + 0.00024415290907648*U[22] - 2.35620151203137e-6*U[29]**3*U[31] - 0.000269281366471626*U[31] + 0.00819565217391304*U[8]
	F[20] = 2.75575081119392*U[0]*U[19] - 0.000888267043268988*U[10]**3*U[20] + 0.250522801017629*U[11]*U[8] - 1.00209120407052*U[14]*U[6] + 4.00836481628207*U[17]*U[3] - 0.0325892314023402*U[19] - 0.0473230535894843*U[20] + 0.00819565217391304*U[9]
	F[21] = 2.30554431681946e-7*U[12] + 2.5547312812995e-7*U[14] + 1.41879650265813e-7*U[17] - 2.30554431681946e-7*U[1] - 4.98594961275655e-7*U[21] - 0.000231547500016414*U[22]*U[25] - 0.000393263214313593*U[22]*U[27] - 0.00077917571434095*U[23]*U[26] - 0.00102174928578672*U[23]*U[28] - 0.00156202678582502*U[24]*U[27] - 2.49430228227486e-5*U[25] - 2.5547312812995e-7*U[3] - 1.41879650265813e-7*U[6]
	F[22] = 1.27676499203106e-7*U[0] - 1.27676499203106e-7*U[11] + 4.43165421807242e-7*U[15] + 1.84356815471813e-7*U[19] - 0.000297563940911795*U[21]*U[25] + 0.00056941247952257*U[21]*U[27] - 1.45521268835226e-6*U[22] - 0.00185518259457354*U[23]*U[25] - 0.00363689261114416*U[24]*U[26] - 2.49312940023328e-5*U[26] - 4.43165421807242e-7*U[4] - 1.84356815471813e-7*U[8]
	F[23] = -4.59275459509595e-7*U[14] - 0.00110122563125871*U[21]*U[26] + 0.0019601816236405*U[21]*U[28] + 0.000385428970940548*U[22]*U[25] - 3.04757790976247e-6*U[23] - 0.00547309138735578*U[24]*U[25] - 2.49117704610167e-5*U[27] + 4.59275459509595e-7*U[3]
	F[24] = 5.09747214298137e-8*U[0] - 5.09747214298137e-8*U[11] - 0.0028417174170155*U[21]*U[27] - 0.000594010608460015*U[22]*U[26] + 0.00259237963074834*U[23]*U[25] - 5.2727039845599e-6*U[24] - 2.48844888171634e-5*U[28]
	F[25] = -5.43087366736076e-7*U[13] + 0.000529111412123567*U[21]*U[22] + 2.49364240037345e-5*U[21] + 0.00146975392256546*U[22]*U[23] + 0.00288071768822831*U[23]*U[24] - 1.03680315467796e-6*U[25] + 5.43087366736076e-7*U[2]
	F[26] = -1.04390781015316e-6*U[16] + 0.00188040062849988*U[21]*U[23] + 0.00423090141412473*U[22]*U[24] + 2.49247013868922e-5*U[22] - 1.99291491029239e-6*U[26] + 1.04390781015316e-6*U[5]
	F[27] = -0.000176149545706673*U[21]*U[22] + 0.00440373864266684*U[21]*U[24] + 2.49051881654425e-5*U[23] - 3.58443843007766e-6*U[27]
	F[28] = -0.000938435679208948*U[21]*U[23] + 2.4877920928742e-5*U[24] - 5.80838943890229e-6*U[28]
	F[29] = 9.12059910499408e-6*U[10]**4 + 4.64878611115251e-5*U[10]**3*U[12] + 1.67415119447806e-5*U[10]**3*U[14] + 9.29757222230503e-6*U[10]**3*U[17] + 0.000521179401993356*U[10] + 0.000664115245477357*U[12] + 0.000239165516524779*U[14] + 0.000132823049095472*U[17] + 1.50119582264922*U[21]*U[35] - 0.750597911324609*U[21]*U[37] - 1.50119582264922*U[22]*U[34] + 1.12589686698692*U[22]*U[36] - 4.50358746794765*U[23]*U[35] + 2.25179373397383*U[23]*U[37] - 2.62709268963613*U[24]*U[34] - 5.62948433493457*U[24]*U[36] + 1.50119582264922*U[25]*U[31] + 2.62709268963613*U[25]*U[33] - 1.50119582264922*U[26]*U[30] + 4.50358746794765*U[26]*U[32] - 1.12589686698692*U[27]*U[31] + 5.62948433493457*U[27]*U[33] + 0.750597911324609*U[28]*U[30] - 2.25179373397383*U[28]*U[32] - 8.14339205803043e-7*U[29]**4 - 0.000260589700996678*U[29] + 5.70773716822425e-5
	F[30] = -3.54406469365901e-21*U[10]**4 - 5.31652508387687e-5*U[10]**3*U[12] - 2.75103051082387e-5*U[10]**3*U[14] - 1.06330501677538e-5*U[10]**3*U[17] - 2.31450148821357e-19*U[10] - 0.000759506950146622*U[12] - 0.000393006100803065*U[14] - 0.000151901390029325*U[17] - 1.21682350103801*U[21]*U[35] + 0.608411750519007*U[21]*U[37] + 2.34182350103802*U[22]*U[34] - 1.28761762577851*U[22]*U[36] + 5.15047050311404*U[23]*U[35] - 2.57523525155702*U[23]*U[37] + 2.12944112681652*U[24]*U[34] + 6.43808812889255*U[24]*U[36] - 2.34182350103802*U[25]*U[31] - 2.12944112681652*U[25]*U[33] + 1.21682350103801*U[26]*U[30] - 5.15047050311404*U[26]*U[32] + 1.28761762577851*U[27]*U[31] - 6.43808812889255*U[27]*U[33] - 0.608411750519007*U[28]*U[30] + 2.57523525155702*U[28]*U[32] + 3.16434347648126e-22*U[29]**4 - 3.25735682321217e-6*U[29]**3*U[30] - 0.000260589700996678*U[30] - 2.21790142793497e-20
	F[31] = 2.78802933171573e-5*U[10]**3*U[11] - 1.54836099362061e-5*U[10]**3*U[15] - 3.09672198724123e-6*U[10]**3*U[19] + 0.000398291670074584*U[11] - 0.000221195408173122*U[15] - 4.42390816346244e-5*U[19] - 1.125*U[21]*U[34] + 0.375*U[21]*U[36] + 1.875*U[23]*U[34] + 2.25*U[24]*U[35] + 1.125*U[25]*U[30] - 1.875*U[25]*U[32] - 2.25*U[26]*U[33] - 0.375*U[27]*U[30] - 3.25735682321217e-6*U[29]**3*U[31] - 0.000260589700996678*U[31] + 4.36192251724445e-5
	F[32] = -5.06294956237001e-22*U[10]**4 - 1.25605469675209e-5*U[10]**3*U[12] + 2.05688778355549e-5*U[10]**3*U[14] - 2.51210939350418e-6*U[10]**3*U[17] - 0.000179437180657833*U[12] + 0.000293842414478535*U[14] - 3.58874361315667e-5*U[17] - 1.90560783367934*U[21]*U[35] + 0.952803916839669*U[21]*U[37] - 1.46939216632066*U[22]*U[34] - 0.304205875259504*U[22]*U[36] + 1.21682350103801*U[23]*U[35] - 0.608411750519007*U[23]*U[37] + 3.33481370893884*U[24]*U[34] + 1.52102937629752*U[24]*U[36] + 1.46939216632066*U[25]*U[31] - 3.33481370893884*U[25]*U[33] + 1.90560783367934*U[26]*U[30] - 1.21682350103801*U[26]*U[32] + 0.304205875259504*U[27]*U[31] - 1.52102937629752*U[27]*U[33] - 0.952803916839669*U[28]*U[30] + 0.608411750519007*U[28]*U[32] + 4.52049068068751e-23*U[29]**4 + 3.61639254455001e-22*U[29]**3*U[30] - 3.25735682321217e-6*U[29]**3*U[32] - 0.000260589700996678*U[32] - 3.16843061133567e-21
	F[33] = 1.11521173268629e-5*U[10]**3*U[11] + 0.000159316668029834*U[11] - 1.875*U[21]*U[36] - 2.25*U[22]*U[35] - 2.625*U[23]*U[34] + 2.625*U[25]*U[32] + 2.25*U[26]*U[31] + 1.875*U[27]*U[30] - 3.25735682321217e-6*U[29]**3*U[33] - 0.000260589700996678*U[33] + 1.74476900689778e-5
	F[34] = 3.64823964199763e-5*U[10]**3*U[13] + 0.000521179401993355*U[13] + 1.125*U[21]*U[31] - 1.125*U[22]*U[30] + 1.875*U[22]*U[32] - 1.875*U[23]*U[31] + 2.625*U[23]*U[33] - 2.625*U[24]*U[32] - 3.25735682321217e-6*U[29]**3*U[34] - 0.000260589700996678*U[34]
	F[35] = 3.64823964199763e-5*U[10]**3*U[16] + 0.000521179401993355*U[16] + 1.5*U[21]*U[32] + 2.25*U[22]*U[33] - 1.5*U[23]*U[30] - 2.25*U[24]*U[31] - 3.25735682321217e-6*U[29]**3*U[35] - 0.000260589700996678*U[35]
	F[36] = -0.375*U[21]*U[31] + 1.875*U[21]*U[33] + 0.375*U[22]*U[30] - 1.875*U[24]*U[30] - 3.25735682321217e-6*U[29]**3*U[36] - 0.000260589700996678*U[36]
	F[37] = -0.75*U[21]*U[32] + 0.75*U[23]*U[30] - 3.25735682321217e-6*U[29]**3*U[37] - 0.000260589700996678*U[37]
	return F

In [ ]:
ten.tensor_dic

In [ ]:
f(0, np.ones(38)) - f_num(0, np.ones(38))

In [ ]:
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
import matplotlib.pyplot as plt

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

In [ ]:
integrator_num = RungeKuttaIntegrator()
integrator_num.set_func(f_num)

In [ ]:
integrator_num = RungeKuttaIntegrator()
integrator_num.set_func(f_test)

In [ ]:
ic = np.array([ 3.84101549e-02, -8.29674554e-03,  3.04587364e-02,  2.80766373e-02,
       -9.14885177e-03, -9.17520676e-04, -1.76115081e-02,  1.32010146e-02,
        1.62515224e-02,  1.08600254e-03,  1.53918671e+00,  4.13205067e-02,
       -9.25169842e-04,  4.01449139e-03,  6.97326597e-03, -9.93383832e-03,
        8.88594931e-03, -6.07097456e-03,  4.34490969e-03,  4.19834122e-03,
       -2.91974161e-03,  1.03085300e-05,  5.98444985e-04, -2.57753313e-05,
        5.22115566e-06, -3.01445438e-05,  3.26249104e-04, -1.92171554e-05,
        1.38469482e-05,  3.17552667e+00,  2.46854576e-03,  1.44249578e-01,
       -5.94828283e-03,  2.34242352e-02, -3.08095487e-03,  9.15501463e-02,
        1.17932987e-03, -4.34659450e-05])

In [ ]:
%%time
integrator.integrate(0., 1000000., 0.1, ic=ic, write_steps=10)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
%%time
integrator_num.integrate(0., 1000000., 0.1, ic=ic, write_steps=10)
reference_time_num, reference_traj_num = integrator_num.get_trajectories()

In [ ]:
%%time
integrator_num.integrate(0., 1000000., 0.1, ic=ic, write_steps=10)
reference_time_test, reference_traj_test = integrator_num.get_trajectories()

In [ ]:
varx = 22
vary = 31
plt.figure(figsize=(10, 8))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_num[varx], reference_traj_num[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_test[varx], reference_traj_test[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(reference_traj[varx, :])
plt.plot(reference_traj_num[varx, :])
plt.plot(reference_traj_test[varx, :])
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(reference_traj_num[vary, :])
plt.plot(reference_traj[vary, :])
plt.plot(reference_traj_test[vary, :])
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(reference_traj_num[varx, ::1000])
plt.plot(reference_traj[varx, ::1000])
plt.plot(reference_traj_test[varx, ::1000])
plt.show()

In [ ]:
tendencies_sym = np.empty((38, reference_time[::1000].shape[0]))
for n, x in enumerate(reference_time[::1000]):
    x = reference_traj_num[:, n]
    tendencies_sym[:, n] = f(0, x)

In [ ]:
tendencies_sym.shape

In [ ]:
tendencies_num = np.empty_like(tendencies_sym)
for n, x in enumerate(reference_time[::1000]):
    x = reference_traj_num[:, n]
    tendencies_num[:, n] = f_num(0, x)

In [ ]:
tendencies_err = tendencies_sym - tendencies_num

In [ ]:
plt.figure(figsize=(12, 8))
varx = 10
plt.plot(tendencies_err[:, :].T)

plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
varx = [1, 12]
plt.plot(tendencies_err[varx, :].T)

plt.show()